<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


# Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [8]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Iterate over the launch sites and add Circle + Marker
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    
    # Add a circle marker
    circle = folium.Circle(
        coordinate,
        radius=1000,
        color='#d35400', 
        fill=True
    ).add_child(folium.Popup(row['Launch Site']))
    
    # Add a text label
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{row["Launch Site"]}</b></div>'
        )
    )
    
    # Add both to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.



---

### **1. Are all launch sites in proximity to the Equator line?**

**The Equator** is at **0° latitude**.

Let’s look at the latitudes of the known SpaceX launch sites from your dataset (`launch_sites_df`):

| Launch Site                                | Latitude   |
| ------------------------------------------ | ---------- |
| CCAFS LC-40 (Cape Canaveral)               | \~28.56° N |
| KSC LC-39A (Kennedy Space Center)          | \~28.57° N |
| VAFB SLC-4E (Vandenberg Air Force Base)    | \~34.63° N |
| STLS (South Texas Launch Site, Boca Chica) | \~25.99° N |

#### ✅ Answer:

> **No**, not all launch sites are *in close proximity* to the Equator.
> However, they are located in the **southern half of the United States**, which is **relatively closer** to the Equator than, say, Canada or Europe.
> Being nearer the Equator provides **greater rotational velocity of Earth**, which helps rockets reach orbit with less fuel — but SpaceX optimizes using U.S. coastal locations for policy, infrastructure, and trajectory reasons.

---

### **2. Are all launch sites in very close proximity to the coast?**

Let’s examine:

* **Cape Canaveral** (CCAFS) and **Kennedy Space Center** (KSC) — Right next to the **Atlantic Ocean**.
* **Vandenberg (VAFB)** — Right on the **Pacific coast** of California.
* **Boca Chica / STLS** — Right on the **Gulf of Mexico** coast in Texas.

If you visualize this on the `Folium` map and even zoom in, you’ll see that **all of them are within a few kilometers from the ocean**.

#### ✅ Answer:

> **Yes**, all SpaceX launch sites are in **very close proximity to a coast**.
> This is intentional:
>
> * It allows rocket stages to safely land over water.
> * Reduces risk to population.
> * Offers flexibility in orbital launch directions (e.g., eastward launches over the Atlantic).

---

###  Summary:

| Question                           | Answer                                                                   |
| ---------------------------------- | ------------------------------------------------------------------------ |
| Are launch sites near the Equator? | ❌ Not really — they’re closer than most places but not truly equatorial. |
| Are launch sites near the coast?   | ✅ Yes, all are within a few kilometers of the coast.                     |



# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [10]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [11]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

In [12]:
# Create a new column to store marker colors based on launch success/failure
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# View the last few rows to verify
spacex_df[['Launch Site', 'Lat', 'Long', 'class', 'marker_color']].tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [13]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
# Iterate through each launch record
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    
    # Extract launch coordinates
    launch_coord =[record['Lat'],record['Long']]

    # Create marker with color indicating success or failure
    marker = folium.Marker(
        location=launch_coord,
        icon=folium.Icon(color=record['marker_color']),
        popup=folium.Popup(f"Launch Site: {record['Launch Site']}<br>Success: {record['class']}", max_width=250)
    )
    # Add the marker to the cluster
    marker_cluster.add_child(marker)

# Display the map
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [14]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [16]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [17]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

In [18]:
# Choose a launch site (e.g., CCAFS LC-40)
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Find the closest coastline point using MousePosition
coastline_lat = 28.56397
coastline_lon = -80.56809

# Calculate the distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance to coastline: {distance_coastline:.2f} km")

# Visualize the distance with a marker on the map
# Create a folium.Marker on the coastline point
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0.0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
    )
)


# Add marker to map
site_map.add_child(distance_marker)

Distance to coastline: 0.92 km


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [19]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
#site_map.add_child(lines)

In [20]:
# Define coordinates
launch_site_coord = [28.562302, -80.577356]
coastline_coord = [28.56397, -80.56809]

# Create polyline
lines = folium.PolyLine(
    locations=[launch_site_coord, coastline_coord],
    weight=1,
    color='blue'
)

# Add to map
site_map.add_child(lines)

# Display the map
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [21]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [22]:
# Distance between KSC LC-39A and train
# Choose a launch site
launch_site_lat = 28.57323
launch_site_lon = -80.6469

# Find the closest railway point using MousePosition
train_lat = 28.57273
train_lon = -80.65395

# Calculate the distance
distance_train = calculate_distance(launch_site_lat, launch_site_lon, train_lat, train_lon)
print(f"Distance from KSC LC-39A to railway: {distance_coastline:.2f} km")

# Visualize the distance with a marker on the map
# Create a folium.Marker on the coastline point
distance_marker = folium.Marker(
    [train_lat, train_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0.0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_train),
    )
)


# Add marker to map
site_map.add_child(distance_marker)

launch_site_coord = [28.57323, -80.6469]
train_coord = [28.57273, -80.65395]

# Create polyline
lines = folium.PolyLine(
    locations=[launch_site_coord, train_coord],
    weight=1,
    color='green'
)

# Add to map
site_map.add_child(lines)

# Display the map
site_map

Distance from KSC LC-39A to railway: 0.92 km


In [23]:
# Distance between KSC LC-39A and highway
# Choose a launch site
launch_site_lat = 28.57323
launch_site_lon = -80.6469

# Find the closest highway point using MousePosition
highway_lat = 28.57286
highway_lon = -80.65548

# Calculate the distance
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)
print(f"Distance from KSC LC-39A to highway: {distance_highway:.2f} km")

# Visualize the distance with a marker on the map
# Create a folium.Marker on the highway point
distance_marker = folium.Marker(
    [highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0.0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_highway),
    )
)


# Add marker to map
site_map.add_child(distance_marker)

launch_site_coord = [28.57323, -80.6469]
highway_coord = [28.57286, -80.65548]

# Create polyline
lines = folium.PolyLine(
    locations=[launch_site_coord, highway_coord],
    weight=1,
    color='red'
)

# Add to map
site_map.add_child(lines)

# Display the map
site_map

Distance from KSC LC-39A to highway: 0.84 km


In [24]:
# Distance between KSC LC-39A and village Titusville
# Choose a launch site
launch_site_lat = 28.57323
launch_site_lon = -80.6469

# Find the closest Titusville point using MousePosition
titusville_lat = 28.57095
titusville_lon = -80.79998

# Calculate the distance
distance_titusville = calculate_distance(launch_site_lat, launch_site_lon, titusville_lat, titusville_lon)
print(f"Distance from KSC LC-39A to Titusville: {distance_titusville:.2f} km")

# Visualize the distance with a marker on the map
# Create a folium.Marker on the highway point
distance_marker = folium.Marker(
    [titusville_lat, titusville_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0.0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_titusville),
    )
)


# Add marker to map
site_map.add_child(distance_marker)

launch_site_coord = [28.57323, -80.6469]
titusville_coord = [28.57095, -80.79998]

# Create polyline
lines = folium.PolyLine(
    locations=[launch_site_coord, titusville_coord],
    weight=1,
    color='black'
)

# Add to map
site_map.add_child(lines)

# Display the map
site_map

Distance from KSC LC-39A to Titusville: 14.96 km


In [26]:
# Distance between VAFB SLC-4 and railway
# Choose a launch site
launch_site_lat_2 = 34.63338
launch_site_lon_2 = -120.61104


# Find the closest railway point using MousePosition
railway_lat = 34.63355
railway_lon = -120.62486


distance_railway = calculate_distance(launch_site_lat_2, launch_site_lon_2, railway_lat, railway_lon)
print(f"Distance from VAFB SLC-4 to Railway: {distance_railway:.2f} km")



# Create a folium.Marker on the railway point
distance_marker = folium.Marker(
    [railway_lat, railway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0.0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_railway),
    )
)


# Add marker to map
site_map.add_child(distance_marker)

launch_site_coord = [34.63338, -120.61104]
railway_coord = [34.63355, -120.62486]

# Create polyline
lines = folium.PolyLine(
    locations=[launch_site_coord, railway_coord],
    weight=1,
    color='green'
)

# Add to map
site_map.add_child(lines)

# Display the map
site_map

Distance from VAFB SLC-4 to Railway: 1.26 km


In [27]:
# Distance between VAFB SLC-4 and village Lompoc and railway
# Choose a launch site
launch_site_lat_2 = 34.63338
launch_site_lon_2 = -120.61104

# Find the closest Lompoc point using MousePosition
lompoc_lat = 34.63708
lompoc_lon = -120.462


# Calculate the distance
distance_lompoc = calculate_distance(launch_site_lat_2, launch_site_lon_2, lompoc_lat, lompoc_lon)
print(f"Distance from VAFB SLC-4 to Lompoc: {distance_lompoc:.2f} km")


# Visualize the distance with a marker on the map
# Create a folium.Marker on the railway point
distance_marker = folium.Marker(
    [lompoc_lat, lompoc_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0.0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_lompoc),
    )
)



# Add marker to map
site_map.add_child(distance_marker)

launch_site_coord = [34.63338, -120.61104]
lompoc_coord = [34.63708, -120.462]

# Create polyline
lines = folium.PolyLine(
    locations=[launch_site_coord, lompoc_coord],
    weight=1,
    color='black'
)

# Add to map
site_map.add_child(lines)

# Display the map
site_map

Distance from VAFB SLC-4 to Lompoc: 13.65 km


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


1.   Are launch sites in close proximity to railways?
* KSC LC-39A. Distance from KSC LC-39A to railway: 0.92 km
  
2.   Are launch sites in close proximity to highways?
* KSC LC-39A. Distance from KSC LC-39A to highway: 0.84 km

3.   Are launch sites in close proximity to coastline?
* CCAFS LC-40. Distance from CCAFS LC-40 to coastline: 0.92 km

5.   Do launch sites keep certain distance away from cities?
* VAFB SLC-4. Distance from VAFB SLC-4 to Lompoc: 13.65 km


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
